# Imports

In [1]:
import pandas as pd
from collections import Counter

# Prepare Training Data

In [2]:
df = pd.read_json('../data/MMHS12M.json', orient='index', convert_axes=False, convert_dates=False, keep_default_dates=False) #

df.head(5)

,level_0,index,labels,tweet_text,labels_str,majority_label,len_text,tweet_text_removed_stopwords
0,0,1114679353714016256,"[1, 1, 1]",@ Nigga FriskDontMiss: https / t co /. cAsaLWE...,"[Religion, Racist, Homophobe]",1,2,@friskdontmiss nigga https://t.co/casalwepue
1,1,1063020048816660480,"[1, 1, 1]",Are horses my retarded: https / /. t co HYhqc6...,"[OtherHate, OtherHate, OtherHate]",1,4,horses retarded https://t.co/hyhqc6d5wn
2,2,1108927368075374592,"[0, 0, 0]",NIGGA MA “ ON YOUNGBOY MOMMA REAL BE SPITTING ...,"[NotHate, NotHate, NotHate]",0,8,“nigga momma youngboy spitting real shit nigga...
3,3,1114558534635618304,"[1, 0, 0]",RT I xxSuGVNGxx: into HOLY ran this TODAY 😭 😭 ...,"[Racist, NotHate, NotHate]",0,9,rt xxsugvngxx: ran holy nigga today 😭😭😭😭 https...
4,4,1035252480215592960,"[1, 0, 1]",EVERYbody you “ now calling ”! : Nigger https ...,"[Racist, NotHate, Racist]",1,3,“everybody calling nigger now!” https://t.co/6...


Make 1 Column for Classifier

In [3]:
# dct = {0:0, 1:1, 2:1, 3:1, 4:1, 5:1}
# df['labels'] = df['labels'].apply(lambda x: [*map(dct.get, x)])

In [4]:
# df['majority_label'] = df.apply(lambda x: Counter(x['labels']).most_common()[0][0], axis=1)

Delete unnecessary columns

In [5]:
df = df.drop(columns=['labels_str', 'labels'])

Get Spark

In [6]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
import os

MAX_MEMORY = "5g"
spark = SparkSession.builder \
                    .appName('multi_class_text_classifiter')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .getOrCreate()

In [7]:
# Diesen Abschnitt in eine Methode für Spark map auslagern.

import re

def clean_text(x):
    # Nigger > nigger
    s = x.lower()
    # 234# > 234
    s = re.sub(r'[^A-Za-z0-9 ]+', ' ', s)
    # ' 234' > '234'
    s = s.strip()
    
    return (s)

df['tweet_text_clean'] = df['tweet_text'].apply(clean_text)
df['tweet_text_clean']

0             nigga friskdontmiss  https   t co   casalwepue
1          are horses my retarded  https     t co hyhqc6d5wn
2          nigga ma   on youngboy momma real be spitting ...
3          rt i xxsugvngxx  into holy ran this today     ...
4          everybody you   now calling     nigger https  ...
                                 ...                        
1198579    svdate     i just gtconway3d say hes the would...
1198580    nigga cheftime dev congrats on my grinding kee...
1198581     my nigga big  shitty   https   t co   e0snjgbgh9
1198582    did just say   she my to nigga rich     she  a...
1198583    said this joe thanos nigga budden galactic got...
Name: tweet_text_clean, Length: 1198584, dtype: object

In [8]:
from pyspark.sql import SQLContext

# sqlContext = SQLContext(spark)
df_small = df.sample(10000)
cleaned_df = spark.createDataFrame(df_small) #sqlContext

In [9]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer

# 'We hate religion' > 'We' 'hate' 'religion'
tokenizer = Tokenizer(inputCol="tweet_text_clean", outputCol="tokens")
# 'We' > (0.000, 0.032432, ...) 300 Dimensionen
w2v = Word2Vec(vectorSize=300, minCount=0, inputCol="tokens", outputCol="features")
doc2vec_pipeline = Pipeline(stages=[tokenizer,w2v])
doc2vec_model = doc2vec_pipeline.fit(cleaned_df)
doc2vecs_df = doc2vec_model.transform(cleaned_df)

In [10]:
doc2vecs_df.show(1)

+-------+-------------------+--------------------+--------------+--------+----------------------------+--------------------+--------------------+--------------------+
|level_0|              index|          tweet_text|majority_label|len_text|tweet_text_removed_stopwords|    tweet_text_clean|              tokens|            features|
+-------+-------------------+--------------------+--------------+--------+----------------------------+--------------------+--------------------+--------------------+
|  14503|1034908664488964096|Think if is Ronal...|             1|      18|        think ronaldo goo...|think if is ronal...|[think, if, is, r...|[-0.0197205554244...|
+-------+-------------------+--------------------+--------------+--------+----------------------------+--------------------+--------------------+--------------------+
only showing top 1 row



# Model Training

In [11]:
w2v_train_df, w2v_test_df = doc2vecs_df.randomSplit([0.8, 0.2])

In [12]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf_classifier = RandomForestClassifier(labelCol="majority_label", featuresCol="features")

rf_classifier_pipeline = Pipeline(stages=[rf_classifier])
rf_predictions = rf_classifier_pipeline.fit(w2v_train_df).transform(w2v_test_df)

rf_model_evaluator = MulticlassClassificationEvaluator(
    labelCol="majority_label", predictionCol="prediction", metricName="accuracy")

In [13]:
accuracy = rf_model_evaluator.evaluate(rf_predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.748637


In [14]:
from pyspark.ml.classification import LogisticRegression

lr_classifier = LogisticRegression(family="multinomial", labelCol="majority_label", featuresCol="features")

lr_classifier_pipeline = Pipeline(stages=[lr_classifier])
lr_predictions = lr_classifier_pipeline.fit(w2v_train_df).transform(w2v_test_df)

lr_model_evaluator = MulticlassClassificationEvaluator(
    labelCol="majority_label", predictionCol="prediction", metricName="accuracy")

In [15]:
accuracy = lr_model_evaluator.evaluate(lr_predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.75062


In [16]:
lr_predictions.show(10)

+-------+-------------------+--------------------+--------------+--------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|level_0|              index|          tweet_text|majority_label|len_text|tweet_text_removed_stopwords|    tweet_text_clean|              tokens|            features|       rawPrediction|         probability|prediction|
+-------+-------------------+--------------------+--------------+--------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   8011|1056376585593401344|@ fucking u Dodge...|             1|      14|        @dodgers u fuckin...|fucking u dodgers...|[fucking, u, dodg...|[-0.0160603587049...|[0.30421784941022...|[0.64758388130644...|       0.0|
|  10145|1106849844302438400|Show you lit dont...|             0|      18|        lit show dont wan...|show you lit dont